In [2]:
import numpy as np

tbl - таблица, элементы которой соответствуют степеням примитивного элемента $\beta$ поля $GF(2^2)$, построенного по модулю неприводимого многочлена $x^2 + x + 1$. Таким образом $\beta^i = tbl[i], i=0..3$ 

In [3]:
tbl = np.array([
    [0, 1],
    [1, 0],
    [1, 1]
])

In [4]:
# получаем степень  beta, представление элемента через генератор поля
def get_deg(a):
    if all(a == 0):
        return None
    for i in range(len(tbl)):
        if all(tbl[i] == a):
            return i

# Произведение 2 элементов в GF(2 ^ 2) через экспоненциальное представление
def mul2(a, b):
    if (all(a == 0) or all(b == 0)):
        return np.zeros((2,)).astype(int)
    return tbl[(get_deg(a) + get_deg(b)) % 3]

# Сумма 2 елементов в GF(2 ^ 2)
def sum2(a, b):
    return a ^ b

# Деление 2 элементов в GF(2 ^ 2) через экспоненциальное представление
def div2(a, b):
    if (all(a == 0) or all(b == 0)):
        raise ValueError
    return tbl[(get_deg(a) - get_deg(b)) % 3]

In [7]:
# функция деления многочлена на многочлен столбиком с коэффициентами в поле GF(2^2)
def poly_div(poly, div):
    if len(div) > len(poly):
        raise ValueError
    answ = [0] * len(poly)

    p_copy = poly.copy()
    while len(p_copy) >= len(div):
        ind = len(p_copy) - len(div)
        answ[ind] = div2(p_copy[0], div[0])

        s = [mul2(answ[ind], div[i]) for i in range(len(div))]
        for i in range(ind):
            s.append([0, 0])

        s = np.array(s)
        for i in range(len(p_copy)):
            p_copy[i] = sum2(p_copy[i], s[i])

        i = 0
        while i < len(p_copy) and all(p_copy[i] == 0):
            i += 1

        if i == len(p_copy):
            return np.zeros((2, ))
        else:
            p_copy = p_copy[i:]

    return p_copy

Будем представлять каждый полином, как вектор кэффициентов, т.е. $a_n x^n + a_{n-1} x^{n - 1} + ... + a_1 x + a_0$ представим в виде массива $[a_n, a_{n-1}, ..., a_1, a_0]$. Так как мы ищем полином 2 степени вида $x^2 + sx + k; s, k \in GF(2^2)$, неприводимый в $GF(2^2)$, то сосздадим массив polynomials всех полиномов степени 1, которые могут являться делителями полинома $x^2 + sx + k$.

In [8]:
polynomials = []

for i in range(3):
    polynomials.append([tbl[i], np.zeros((2, )).astype(int)])

for i in range(3):
    for j in range(3):
        polynomials.append([tbl[i], tbl[j]])

polynomials = np.array(polynomials)

Переберем все возможные $s, k \in GF(2^2)$ и найдем подходящие полиномы вида $x^2 + sx + k$.

In [9]:
for s in tbl:
    for k in tbl:
        poly = np.array([tbl[0], s, k])
        for div in polynomials:
            if all(poly_div(poly, div).ravel() == 0):
                break
        else:
            print(f'Полином неприводимый при s = {s}   k={k}')

Полином неприводимый при s = [0 1]   k=[1 0]
Полином неприводимый при s = [0 1]   k=[1 1]
Полином неприводимый при s = [1 0]   k=[0 1]
Полином неприводимый при s = [1 0]   k=[1 0]
Полином неприводимый при s = [1 1]   k=[0 1]
Полином неприводимый при s = [1 1]   k=[1 1]


Найдем остатки от деления полинома $x^2 + [0, 1]x + [1, 0]$ на все полиномы 1 степени с коэффициентами в $GF(2^2)$ и удостоверимся, что он неприводимый.

In [11]:
poly = np.array([tbl[0], tbl[0], tbl[1]])
for div in polynomials:
    print(f'остаток от деления на {div.tolist()} = {poly_div(poly, div)}\n')

остаток от деления на [[0, 1], [0, 0]] = [[1 0]]

остаток от деления на [[1, 0], [0, 0]] = [[1 0]]

остаток от деления на [[1, 1], [0, 0]] = [[1 0]]

остаток от деления на [[0, 1], [0, 1]] = [[1 0]]

остаток от деления на [[0, 1], [1, 0]] = [[1 1]]

остаток от деления на [[0, 1], [1, 1]] = [[1 1]]

остаток от деления на [[1, 0], [0, 1]] = [[1 1]]

остаток от деления на [[1, 0], [1, 0]] = [[1 0]]

остаток от деления на [[1, 0], [1, 1]] = [[1 1]]

остаток от деления на [[1, 1], [0, 1]] = [[1 1]]

остаток от деления на [[1, 1], [1, 0]] = [[1 1]]

остаток от деления на [[1, 1], [1, 1]] = [[1 0]]



Таким образом, полином $x^2 + [0, 1]x + [1, 0]$ явлется неприводимым в $GF(2^2)$, и мы по можем строить поле $GF((2^2)^2)$ по модулю $x^2 + [0, 1]x + [1, 0]$.